In [16]:
import re
import subprocess
import numpy as np
import pickle
import os
class lkh:
    def __init__(self , dist_mat_pic_file , stop_time = 300):
        '''
        Note : This code has to be in the same directory as the distance matrices and the mapping file.

        Input : takes the name of the distance matrix pickle file , and the stopping criteria in minutes.

        Returns : length_of_tour , sequence of cities  , number of cities
        '''
        self.dist_mat_pic_file = dist_mat_pic_file
        self.dist_mat_file_name = dist_mat_pic_file.split(".")[0]
        self.stop_time = stop_time
        print(self.dist_mat_file_name)

    def solve(self):
        '''
        solves the TSP instance using .exe file
        '''
        # Generate the .par file
        self.generate_parameter_file(self.dist_mat_file_name)

        # Generate the .atsp file
        dist_mat = self.read_pickle(self.dist_mat_pic_file)
        self.generate_atsp_file(dist_mat , self.dist_mat_file_name)

        try :    
            
            #first check if the directory exists
            filename = f"{self.dist_mat_file_name}_start.txt"

            # Get the absolute path of the file
            file_path = os.path.abspath(filename)

            # Check if the file exists
            if os.path.exists(file_path):
                # if starting solution file exists
                print("Starting solution found!")
                result = subprocess.check_output(["LKH-2.exe", f"{self.dist_mat_file_name}.par", "-S" , f"{self.dist_mat_file_name}_start.txt" ], stdin=subprocess.DEVNULL , timeout = self.stop_time)
            else:
                # if starting  solutions file does not exist
                print("No starting solution found!")
                result = subprocess.check_output(["LKH-2.exe", f"{self.dist_mat_file_name}.par"], stdin=subprocess.DEVNULL , timeout = self.stop_time)
        
        except subprocess.TimeoutExpired:
            print("Execution timed out. Terminating the process.")
            # print(output)
        
        result_file_name = self.dist_mat_file_name + '.txt'
        print("Result is saved in : ",result_file_name)
        length , tour_section  , cities = self.get_obj_val_and_tour(result_file_name)

        # print("Objective function value : ",length)
        # print("The routing : " , tour_section)
        # print("No of cities : " , cities)

        return length , tour_section  , cities


    
    def get_obj_val_and_tour(self , tour_file):
        """
        Extracts the length and tour section from a tour file.

        Args:
            tour_file (str): Path to the tour file.

        Returns:
            tuple: A tuple containing the length and tour section as a list.
        """
        length = None
        dimension = None
        tour_section = []

        if(os.path.exists(tour_file)):
            with open(tour_file, "r") as file:
                for line in file:
                    line = line.strip()

                    if line.startswith("COMMENT : Length"):
                        length = int(line.split("=")[1].strip())
                    elif line.startswith("DIMENSION"):
                        dimension = int(line.split(":")[1].strip())
                    elif line == "TOUR_SECTION":
                        # Start reading the tour section
                        for tour_line in file:
                            tour_line = tour_line.strip()
                            if tour_line == "-1":
                                # End of tour section
                                break
                            tour_section.append(int(tour_line))

            return length, tour_section , dimension
        else:
            return 0 , 0 , 0
 
    
    def get_routing(self , filename):
        '''
        returns the routing in a numpy array
        '''
        start_keyword = 'TOUR_SECTION'
        end_keyword = '-1'
        with open(filename, 'r') as file:
            lines = []
            read_flag = False
            for line in file:
                if start_keyword in line:
                    read_flag = True
                    continue  # skip this line
                if read_flag:
                        if end_keyword in line:
                            read_flag = False
                            break
                        else:
                            lines.append(line.strip())

        lines = np.array(lines)
        return lines
    
    def generate_parameter_file(self , filename):
        '''
        Generates the .par file which is a required input for our .exe file.
        '''
        runs = 1
        filename_atsp = filename + ".atsp"
        filename_txt = filename + ".txt"  
        filename_par = filename + ".par"

        with open(filename_par, 'w') as f:
            f.write(f"PROBLEM_FILE = {filename_atsp}\n")
            f.write(f"OUTPUT_TOUR_FILE = {filename_txt}\n")
            f.write(f"OPTIMUM = 40634081 \n")
            f.write(f"INITIAL_PERIOD = 1000 \n")
            f.write(f"MAX_CANDIDATES = 6 \n")
            f.write(f"MAX_TRIALS = 1000 \n")
            f.write(f"MOVE_TYPE = 6 \n")
            f.write(f"PATCHING_C = 6 \n")
            f.write(f"PATCHING_A = 5 \n")
            f.write(f"RECOMBINATION = GPX2 \n")
            f.write(f"RUNS = {runs} \n")

    
    def round_up(self , lst):
        '''
        input : list of lists having non integral values
        output : list of lists having integral values.
        Function to round up numbers and return as integers
        '''
        rounded_lst = []
        for inner_lst in lst:
            rounded_inner_lst = []
            for num in inner_lst:
                rounded_num = int(round(num))
                rounded_inner_lst.append(rounded_num)
            rounded_lst.append(rounded_inner_lst)
        return rounded_lst  


    def read_pickle(self,pickle_path):
        # Open the pickle file in read-binary mode
        with open(pickle_path, 'rb') as f:
            # Load the object from the pickle file
            pic_obj = pickle.load(f)
        
        pic_obj = self.round_up(pic_obj)
        return pic_obj
    
    def generate_atsp_file(self , dist_mat, filename):   
        '''
        input : 
        a symmentric matrix  
        directory of a .tsp file you want to generate
        number of cities in the TSP

        Output : 
        a .tsp file generated in the filename directory
        '''
        # Defining the number of cities
        num_cities = len(dist_mat)

        # Write the coordinates to a .atsp file
        # fname = filename.split("/")[-1].split(".")[0]
        fname = filename 
        with open(filename+".atsp", 'w') as f:
            f.write(f"NAME: {fname}\n")
            # f.write("COMMENT: Randomly generated TSP instance\n")
            f.write("TYPE: TSP (M.~Hofmeister)\n")
            f.write(f"DIMENSION: {num_cities}\n") #becase the distance matrix doubles in size when converted to a symmentric instance
            f.write("EDGE_WEIGHT_TYPE: EXPLICIT\n")
            f.write("EDGE_WEIGHT_FORMAT: FULL_MATRIX\n")
            # f.write("NODE_COORD_SECTION\n")
            f.write("DISPLAY_DATA_TYPE: NO_DISPLAY\n")
            f.write("EDGE_WEIGHT_SECTION\n")
            for i in range(len(dist_mat)):
                for j in range(len(dist_mat)):
                    f.write(f'{str(dist_mat[i][j])}\t')
                f.write("\n")
            f.write("EOF\n")

    

    


        

In [17]:
import glob

# List all files in the current directory that end with ".p"
file_list = glob.glob("*.p")

new_file_list = file_list[0:10]
print(new_file_list)

time_list = [ 60 , 180 , 300]

arr = []
for file in new_file_list:
    data_pt = []
    count = 0
    for t in time_list:    
        s1 = lkh(file , t)
        length , tour_section  , cities = s1.solve()
        if count == 0 :
            data_pt.append(file)
            data_pt.append(cities)
            data_pt.append(length)
            count+=1
        else:
            data_pt.append(length)
    
    arr.append(data_pt)


for i in arr:
    print(i)
    

['dm_Warmsen_Friday_31600-01_scenario_high_all.p', 'dm_Warmsen_Friday_31600-01_scenario_high_letter.p', 'dm_Warmsen_Friday_31600-01_scenario_high_others.p', 'dm_Warmsen_Friday_31600-01_scenario_high_package.p', 'dm_Warmsen_Friday_31600-01_scenario_low_all.p', 'dm_Warmsen_Friday_31600-01_scenario_low_letter.p', 'dm_Warmsen_Friday_31600-01_scenario_low_others.p', 'dm_Warmsen_Friday_31600-01_scenario_low_package.p', 'dm_Warmsen_Friday_31600-01_scenario_medium_all.p', 'dm_Warmsen_Friday_31600-01_scenario_medium_letter.p']
dm_Warmsen_Friday_31600-01_scenario_high_all
No starting solution found!
Execution timed out. Terminating the process.
Result is saved in :  dm_Warmsen_Friday_31600-01_scenario_high_all.txt
dm_Warmsen_Friday_31600-01_scenario_high_all
No starting solution found!
Execution timed out. Terminating the process.
Result is saved in :  dm_Warmsen_Friday_31600-01_scenario_high_all.txt
dm_Warmsen_Friday_31600-01_scenario_high_all
No starting solution found!
Execution timed out. Te

In [19]:
import pandas as pd
row1 = ['file_name' , 'no_of_cities' , 'obj@60', 'obj@180', 'obj@300']
df = pd.DataFrame(arr , columns = row1)
df.head(10)
# Convert DataFrame to CSV file
df.to_csv('output.csv', index=False)

In [17]:
#Getting all the distance matricecs in a single directory.
import os
import shutil

source_directory = 'C:/Users/akshay/Documents/analytics project/AOP_DP_Analytics/AOP_DP_Analytics/AnalyticsProject/Warmsen_data/Warmsen'
dest_dir = 'C:/Users/akshay/Documents/analytics project/AOP_DP_Analytics/AOP_DP_Analytics/AnalyticsProject/Warmsen_data'
count = 0
# Iterate over all files and directories in the source directory
for root, dirs, files in os.walk(source_directory):
    for file in files:
        if file.endswith('.p'):
            source_file = os.path.join(root, file)
            print(file)
            count+=1
            destination_file = os.path.join(dest_dir, file)  # Paste in the current directory
            shutil.copy2(source_file, destination_file)
# print(count)

dm_Warmsen_Friday_31600-01_scenario_high_all.p
map_Warmsen_Friday_31600-01_scenario_high_all.p
dm_Warmsen_Friday_31600-01_scenario_low_all.p
map_Warmsen_Friday_31600-01_scenario_low_all.p
dm_Warmsen_Friday_31600-01_scenario_medium_all.p
map_Warmsen_Friday_31600-01_scenario_medium_all.p
dm_Warmsen_Friday_31600-01_scenario_high_letter.p
map_Warmsen_Friday_31600-01_scenario_high_letter.p
dm_Warmsen_Friday_31600-01_scenario_low_letter.p
map_Warmsen_Friday_31600-01_scenario_low_letter.p
dm_Warmsen_Friday_31600-01_scenario_medium_letter.p
map_Warmsen_Friday_31600-01_scenario_medium_letter.p
dm_Warmsen_Friday_31600-01_scenario_high_others.p
map_Warmsen_Friday_31600-01_scenario_high_others.p
dm_Warmsen_Friday_31600-01_scenario_low_others.p
map_Warmsen_Friday_31600-01_scenario_low_others.p
dm_Warmsen_Friday_31600-01_scenario_medium_others.p
map_Warmsen_Friday_31600-01_scenario_medium_others.p
dm_Warmsen_Friday_31600-01_scenario_high_package.p
map_Warmsen_Friday_31600-01_scenario_high_package.p
